In [1]:
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, joblib
import glob
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import time

In [3]:
train_csv = pd.read_csv('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data.csv')

In [5]:
train_csv

,Patient ID,Recording locations:,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Most audible location,...,Systolic murmur pitch,Systolic murmur quality,Diastolic murmur timing,Diastolic murmur shape,Diastolic murmur grading,Diastolic murmur pitch,Diastolic murmur quality,Outcome,Campaign,Additional ID
0,2530,AV+PV+TV+MV,Child,Female,98.0,15.9,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
1,9979,AV+PV+TV+MV,Child,Female,103.0,13.1,False,Present,AV+MV+PV+TV,TV,...,High,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
2,9983,AV+PV+TV+MV,Child,Male,115.0,19.1,False,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
3,13918,AV+PV+TV+MV,Child,Male,98.0,15.9,False,Present,TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
4,14241,AV+PV+TV+MV,Child,Male,87.0,11.2,False,Present,AV+MV+PV+TV,PV,...,Low,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,85340,AV+PV+TV+MV,Child,Male,105.0,16.6,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN
938,85341,AV+PV+TV+MV,Child,Male,92.0,15.2,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN
939,85343,AV+PV+TV+MV,Child,Female,97.0,13.5,False,Present,MV+TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
940,85345,AV+PV,Child,Female,132.0,38.1,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN


In [36]:
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942 entries, 0 to 941
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Patient ID                942 non-null    int64  
 1   Recording locations:      942 non-null    object 
 2   Age                       868 non-null    object 
 3   Sex                       942 non-null    object 
 4   Height                    826 non-null    float64
 5   Weight                    837 non-null    float64
 6   Pregnancy status          942 non-null    bool   
 7   Murmur                    942 non-null    object 
 8   Murmur locations          179 non-null    object 
 9   Most audible location     179 non-null    object 
 10  Systolic murmur timing    178 non-null    object 
 11  Systolic murmur shape     178 non-null    object 
 12  Systolic murmur grading   178 non-null    object 
 13  Systolic murmur pitch     178 non-null    object 
 14  Systolic m

총 942명 환자

In [15]:
train_data_folder = '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/'
valid_data_folder = '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation/'
model_folder = '/home/jh20/narin/physionet/python-classifier-2022/model'

In [19]:
train_wav_file = glob.glob(train_data_folder + '*.wav')
valid_wav_file = glob.glob(valid_data_folder + '*.wav')

In [8]:
len(train_wav_file)

2532

In [20]:
len(valid_wav_file)

631

In [11]:
absent_id = list(train_csv[train_csv['Murmur']== 'Absent'].loc[:,'Patient ID'])
unknown_id = list(train_csv[train_csv['Murmur']== 'Unknown'].loc[:,'Patient ID'])

In [23]:
train_csv[train_csv['Murmur']== 'Unknown'].iloc[:,9].value_counts()

Series([], Name: Most audible location, dtype: int64)

In [15]:
# murmur = present 
pv_id = list(train_csv[train_csv.iloc[:,9]== 'PV'].loc[:,'Patient ID'])
tv_id = list(train_csv[train_csv.iloc[:,9]== 'TV'].loc[:,'Patient ID'])
mv_id = list(train_csv[train_csv.iloc[:,9]== 'MV'].loc[:,'Patient ID'])
av_id = list(train_csv[train_csv.iloc[:,9]== 'AV'].loc[:,'Patient ID'])

In [16]:
absent_train, absent_test, _, _ = train_test_split(absent_id, absent_id, test_size=0.2, random_state=777)#, stratify=y_data) 
unknown_train, unknown_test, _, _ = train_test_split(unknown_id, unknown_id, test_size=0.2, random_state=666)#, stratify=y_data)

In [17]:
pv_train, pv_test, _, _ = train_test_split(pv_id, pv_id, test_size=0.2, random_state=666)#, stratify=y_data) 
tv_train, tv_test, _, _ = train_test_split(tv_id, tv_id, test_size=0.2, random_state=666)#, stratify=y_data)
mv_train, mv_test, _, _ = train_test_split(mv_id, mv_id, test_size=0.2, random_state=666)#, stratify=y_data)
av_train, av_test, _, _ = train_test_split(av_id, av_id, test_size=0.2, random_state=666)#, stratify=y_data)

In [24]:
print(len(absent_train)+len(unknown_train)+len(pv_train)+len(tv_train)+len(mv_train)+len(av_train))
print(len(absent_test)+len(unknown_test)+len(pv_test)+len(tv_test)+len(mv_test)+len(av_test))

751
191


In [25]:
all_train = list()
all_test = list()

all_train.extend(absent_train)
all_train.extend(unknown_train)
all_train.extend(pv_train)
all_train.extend(tv_train)
all_train.extend(mv_train)
all_train.extend(av_train)

all_test.extend(absent_test)
all_test.extend(unknown_test)
all_test.extend(pv_test)
all_test.extend(tv_test)
all_test.extend(mv_test)
all_test.extend(av_test)

In [27]:
print(len(set(all_train)))
print(len(set(all_test)))

751
191


In [29]:
all_file = glob.glob(data_folder + '*.*')
print(len(all_file))

10432


In [30]:
all_file

['/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49873_AV.tsv',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68682_AV.hea',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49963_AV.hea',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50379_PV.tsv',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84764_AV.hea',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85057_AV.tsv',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68213_PV.hea',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84780_AV.wav',
 '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.or

In [31]:
fnm_id = list()
for i in all_file:
    ids = i.split('/')[-1][:5]
    fnm_id.append(ids)
print(len(fnm_id))
print(len(set(fnm_id)))

10432
946


In [37]:
all_train

[50771,
 50151,
 50738,
 85340,
 84992,
 84994,
 85213,
 50773,
 50756,
 50263,
 50216,
 50379,
 50222,
 50772,
 84935,
 50754,
 69188,
 49999,
 78592,
 85143,
 50805,
 50057,
 50385,
 50247,
 50753,
 50074,
 50053,
 49678,
 49995,
 85250,
 69106,
 84775,
 61610,
 84976,
 85287,
 49661,
 49959,
 50802,
 50668,
 84839,
 50707,
 50740,
 50032,
 50278,
 84890,
 84876,
 68394,
 50661,
 50812,
 49561,
 85102,
 85161,
 84936,
 84884,
 49987,
 49979,
 50015,
 85018,
 50306,
 85265,
 84721,
 84985,
 85252,
 85315,
 84805,
 49960,
 85109,
 84790,
 84861,
 85262,
 50092,
 49853,
 50009,
 84807,
 50751,
 85066,
 50689,
 85249,
 85114,
 49610,
 49970,
 84946,
 50723,
 84699,
 85044,
 84697,
 50646,
 85316,
 84977,
 50103,
 50317,
 50687,
 50018,
 84942,
 81297,
 49842,
 85075,
 85053,
 50048,
 49568,
 84753,
 50125,
 50146,
 84824,
 84970,
 50732,
 49946,
 84887,
 50748,
 49577,
 68755,
 50762,
 68292,
 14998,
 84815,
 85011,
 50261,
 84933,
 85244,
 84835,
 85176,
 85027,
 50752,
 50677,
 84838,


In [39]:
import shutil
import time
for file in all_file :
    for ids in all_train :
        ids = str(ids)
        if file.split('/')[-1].startswith(ids) :
            print(file)            
            shutil.copyfile(file, '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/'+ file.split('/')[-1])
          
            

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49873_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49963_AV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50379_PV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84764_AV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85057_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68213_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84780_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84949_PV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/t

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68646_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50318_MV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/69176_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85250_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85293_MV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/46778.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68260.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84769_MV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/trainin

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68874_MV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50263_MV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85269_PV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84706_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50027.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84887_TV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85112_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84868_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/trai

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84950_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49631_MV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84890_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85250_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50751_MV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50122_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85113_TV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84976_MV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/t

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68864_TV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50822_AV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49998_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/29045_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84864_PV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50161_TV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84919_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49630_PV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/t

...

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50100_MV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50334_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50740_AV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50766.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85319_AV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84769_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50113_AV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85110_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/trai

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85086_MV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84742_TV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85028_MV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85057_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84799_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/81297_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49854_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/72288_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/t

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/69176.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84985_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50826_MV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68737.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68470_MV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/64256_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50115_PV_1.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50129_MV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/57706_TV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/78280_AV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50766_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/46579_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84690.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50280_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85122_AV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84976_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/trai

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50123.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50321_MV_1.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50379_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50228_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/9979_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50323_MV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/64715_MV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84918_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/tra

In [45]:
import shutil
for file in all_file :
    for ids in all_test :
        ids =str(ids)
        if file.split('/')[-1].startswith(ids) :
            print(file)
            
            shutil.copyfile(file, '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation/'+ file.split('/')[-1])
          

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68682_AV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85081.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50341.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85162_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84982_PV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84874_MV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68545_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85118_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/trainin

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50168.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85288_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84857_MV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50770_TV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50213_AV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49952_MV_2.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68327_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84896_AV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/tr

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50624_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85225.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84826_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49628_MV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49653_PV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85219_PV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68423_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85128_MV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/trai

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/69060_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68484_TV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49691_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50079_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50299_MV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49761_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85084_PV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50067_TV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/t

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84865_PV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/69125_TV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85247_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85225_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85279_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49641_MV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84760_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49691_TV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/t

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85121_MV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85181_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/69147_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85219_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50693_TV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/69144_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84822.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49628_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/trai

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85079_MV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85140_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85118_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49641_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50781_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50210_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50275_TV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84961_TV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/t

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85216_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84896_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84973_PV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84896_TV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85043_MV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49641_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85144.txt
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50656_TV_2.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/tr

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/40798_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84990_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84692_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50067_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50685_MV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84852_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/69147_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/84829_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/t

/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/50800_TV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85338_MV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85282_AV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/49628_PV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85305_MV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/85247_MV.hea
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68449_TV.tsv
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/68484_AV.wav
/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/t

In [46]:
validation = glob.glob('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation/*.txt')

In [47]:
len(validation)

191

In [44]:
all_test  

[84961,
 68952,
 85288,
 49832,
 85338,
 85134,
 50800,
 50155,
 85121,
 84814,
 84822,
 84733,
 84758,
 68682,
 50659,
 85084,
 49683,
 50086,
 68698,
 50214,
 85100,
 50815,
 50619,
 50047,
 84874,
 50699,
 85225,
 84760,
 50145,
 70280,
 50644,
 84743,
 49641,
 50218,
 49980,
 50671,
 68404,
 85115,
 84708,
 50375,
 50213,
 50165,
 85118,
 85242,
 50249,
 50012,
 50631,
 50168,
 69112,
 74417,
 85305,
 50006,
 84993,
 85140,
 84986,
 84734,
 85128,
 84852,
 68460,
 38337,
 50386,
 50664,
 68449,
 84917,
 84952,
 50708,
 85162,
 40798,
 50348,
 85181,
 50143,
 68738,
 84711,
 84829,
 84826,
 84879,
 49978,
 84990,
 49896,
 73316,
 85329,
 81035,
 50341,
 84978,
 84923,
 49983,
 50743,
 50067,
 68887,
 85152,
 50164,
 68406,
 50104,
 68545,
 50685,
 84974,
 84856,
 85184,
 50275,
 85166,
 50737,
 59536,
 69144,
 50342,
 68484,
 85216,
 85144,
 84931,
 50641,
 84855,
 50643,
 50105,
 84813,
 49829,
 84920,
 50026,
 50770,
 85167,
 50210,
 85279,
 50255,
 68327,
 50693,
 50219,
 84893,


In [4]:
from torch.utils.data import Dataset,DataLoader, RandomSampler
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import torchaudio
import random
import numpy as np
import librosa
import glob
import math






def read_recording(wav_fn, audio_length= 20, sr= 4000):

    recording, frequency = load_wav_file(wav_fn)


    length = audio_length*sr
    if recording.shape[0] <= length:
        shortage = length - recording.shape[0]
        recording = np.pad(recording, (0, shortage), 'wrap')
    start_frame = np.int64(random.random()*(recording.shape[0]-length))
    recording = recording[start_frame:start_frame + length] 

    return recording



class train_loader(Dataset):
    def __init__(self,data_folder, **kwargs):
        self.data_folder = data_folder
        
        self.wav_files = glob.glob(data_folder + '*.wav')
        self.num_wav_files = len(self.wav_files)
        
        self.murmur_classes = ['Present', 'Unknown', 'Absent']
        self.num_murmur_classes = len(self.murmur_classes)
        self.outcome_classes = ['Abnormal', 'Normal']
        self.num_outcome_classes = len(self.outcome_classes)
        
        self.age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
        self.recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

        
        
        examples = {}
        for filename in self.wav_files:
            current_patient_id = filename.split('/')[-1].split('_')[0]
            current_txt_file = self.data_folder + str(current_patient_id) + '.txt'
            with open(current_txt_file, 'r') as f:
                current_patient_data = f.read()
            current_loc = filename.split('/')[-1].split('.')[0].split('_')[1]
            if filename.split('/')[-1].split('.')[0]  not in examples.keys():
                current_wav_data = filename.split('/')[-1].split('.')[0]
                examples[current_wav_data] = {
                    "id" : current_patient_id,
                    "txt_fn" : current_txt_file,   #txt,
                    "wav_fn" : filename,  #wav
                    "num_location": get_num_locations(current_patient_data),
                    "frequency" : get_frequency(current_patient_data),
                    "location" : current_loc,
                    "age" : get_age(current_patient_data),
                    "sex" : get_sex(current_patient_data),
                    "height" : get_height(current_patient_data),
                    "weight" : get_weight(current_patient_data),
                    "pregnancy_status" : get_pregnancy_status(current_patient_data),
                    "murmur" : get_murmur(current_patient_data),
                    "outcome" : get_outcome(current_patient_data)

                }
        self.examples = examples
        self.examples_list = list(examples.keys())
        
        
        if len(self.examples_list) != self.num_wav_files:
            raise ValueError('length is different!')
    def __len__(self):
        return len(self.examples_list)
    
        
    def __getitem__(self, index):
        
        # patient info
        current_wav_dict = self.examples[self.examples_list[index]]
        current_patient_data = load_patient_data(current_wav_dict["txt_fn"])
        current_recording = read_recording(current_wav_dict["wav_fn"])
        
        #log mel feature
        log_mel_feature = librosa.power_to_db(librosa.feature.melspectrogram(y = current_recording.astype(np.float32),
                                                         sr= 4000,
                                                         n_mels=128,
                                                         n_fft=400, 
                                                         hop_length=128, 
                                                         win_length=400))
        
        # age
        current_patient_age = current_wav_dict["age"]
        current_age_group = np.zeros(6, dtype=np.float32)
        if current_patient_age in self.age_classes:
            j = self.age_classes.index(current_patient_age)
            current_age_group[j] = 1.0
        else :
            current_age_group[5] = 1.0
            
            
        # sex
        sex = current_wav_dict["sex"]
        sex_feature = np.zeros(2, dtype=np.float32)
        if compare_strings(sex, 'Female'):
            sex_feature[0] = 1.0
        elif compare_strings(sex, 'Male'):
            sex_feature[1] = 1.0
            
        # height and weight.
        height = current_wav_dict["height"]
        weight = current_wav_dict["weight"]
        
        ## simple impute
        if math.isnan(height) :
            height = 110.846  #mean
        if math.isnan(weight) :
            weight = 23.767   #mean
            
        height_weight = np.array([height, weight], dtype=np.float32)
        
        
        # Extract pregnancy
        preg_feature = np.zeros(2, dtype=np.float32)
        is_pregnant = current_wav_dict["pregnancy_status"]
        if is_pregnant == True:
            preg_feature[0] = 1.0
        elif is_pregnant == False:
            preg_feature[1] = 1.0

        # Extract location
        location = current_wav_dict["location"]
        num_recording_locations = len(self.recording_locations)
        loc_feature = np.zeros(num_recording_locations, dtype=np.float32)
        if location in self.recording_locations:
            j = self.recording_locations.index(location)
            loc_feature[j] = 1.0


        

        # label
        
        current_murmur = np.zeros(self.num_murmur_classes, dtype=np.float32)
        murmur = get_murmur(current_patient_data)
        if murmur in self.murmur_classes:
            j = self.murmur_classes.index(murmur)
            current_murmur[j] = 1


        current_outcome = np.zeros(self.num_outcome_classes, dtype=np.float32)
        outcome = get_outcome(current_patient_data)
        if outcome in self.outcome_classes:
            j = self.outcome_classes.index(outcome)
            current_outcome[j] = 1
            


        
        return log_mel_feature, current_age_group, sex_feature, height_weight, preg_feature, loc_feature, current_murmur, current_outcome


In [5]:
import argparse, warnings, sys
import torch



parser = argparse.ArgumentParser(description = "HMD_trainer")
## Training Settings
parser.add_argument('--train_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/',     help='train data path')
parser.add_argument('--valid_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation/',     help='validation data path')
parser.add_argument("--lr",default=1e-3, type=float, help="learning rate")
parser.add_argument("--minibatchsize_train", default=128, type=int)
parser.add_argument("--minibatchsize_valid", default=8, type=int)
parser.add_argument("--train_num_workers", default=6, type=int, help="number of training workers")
parser.add_argument("--dev_num_workers", default=4, type=int, help="number of validation workers")
parser.add_argument("--seed", default=617, type=int)
parser.add_argument("--model_path", default='./hmd_CNN', type=str)
parser.add_argument("--logdir", default='./log_hmd/', type=str)
parser.add_argument("--model_name", default='hmd_CNN_model', type=str)
parser.add_argument("--start_iter", default=0, type=int)
parser.add_argument("--end_iter", default=200, type=int)
parser.add_argument("--gpu", default="0", type=str)

warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
args = parser.parse_args(args=[])

In [6]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [7]:
os.makedirs(args.model_path, exist_ok=True)
os.makedirs(args.logdir, exist_ok=True)

In [8]:
open(args.logdir + 'score.txt', "a+")
score_file = open(args.logdir + 'score.txt', "a+")

In [9]:
import logging
def get_logger(filename):
    # Logging configuration: set the basic configuration of the logging system
    log_formatter = logging.Formatter(fmt='%(asctime)s [%(processName)s, %(process)s] [%(levelname)-5.5s]  %(message)s',
                                      datefmt='%m-%d %H:%M')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    # File logger
    file_handler = logging.FileHandler("{}.log".format(filename))
    file_handler.setFormatter(log_formatter)
    file_handler.setLevel(logging.DEBUG)
    logger.addHandler(file_handler)
    # Stderr logger
    std_handler = logging.StreamHandler(sys.stdout)
    std_handler.setFormatter(log_formatter)
    std_handler.setLevel(logging.DEBUG)
    logger.addHandler(std_handler)
    return logger


def cal_indicator(pre,label):
    TP = 0.0
    FN = 0.0
    TN = 0.0
    FP = 0.0
    for i, it in enumerate(pre):
        if it == 1.0 and label[i] == 1.0:
            TP += 1.0
        elif it == 1.0 and label[i] == -0.0:
            FP += 1
        elif it == -0.0 and label[i] == 1.0:
            FN += 1
        elif it == -0.0 and label[i] == -0.0:
            TN += 1.0
    return TP, FP, TN, FN


In [10]:
model_path = args.model_path
log_dir = args.logdir
logger = get_logger(log_dir + '/' + model_path)

In [ ]:
epoch_len = min(
    [
        len(tot_train_data[indx])
        // (
            config["training"]["batch_size"][indx]
            * config["training"]["accumulate_batches"]
        )
        for indx in range(len(tot_train_data))
    ]
)

In [13]:
import sys
sys.path.append('/home/jh20/narin/dcase/DESED_task')
sys.path.append('/home/jh20/narin/dcase/DESED_task/recipes/dcase2022_task4_baseline')

In [30]:
from desed_task.nnet.CRNN_cbam import CRNN

In [38]:
crnn = CRNN()

In [42]:
crnn = crnn.cuda(device)

In [11]:
trainset = train_loader(args.train_data_folder)
validset = train_loader(args.valid_data_folder)
point_sampler = RandomSampler(trainset)

In [20]:
from desed_task.utils.schedulers import ExponentialWarmup

epoch_len = len(trainset)// (
            args.minibatchsize_train
            * 1
        )


opt = torch.optim.Adam(cnn.parameters(), lr=args.lr, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, 0.91)
exp_steps = 50 * epoch_len
exp_scheduler = ExponentialWarmup(opt, 0.001, exp_steps)


# BCE_loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor(1.0)) 


criterion = torch.nn.CrossEntropyLoss()

In [41]:
exp_steps

1950

In [8]:
len(trainset)

2532

In [9]:
len(validset)

631

In [21]:
dataloader_train = DataLoader(dataset=trainset,
                        batch_size=args.minibatchsize_train,
                               sampler =point_sampler,
                               num_workers=args.train_num_workers)
dataloader_valid = DataLoader(dataset=validset,
                        batch_size=args.minibatchsize_valid,
                               num_workers=args.dev_num_workers)

In [43]:
a,b,c,d,e,f,g,h = next(iter(dataloader_valid))

In [45]:
b.shape

torch.Size([1, 6])

In [17]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())

Available devices  1


In [18]:
GPU_NUM = args.gpu # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU

In [19]:
cnn = CNN()
cnn = cnn.cuda(device)

In [47]:
import tqdm
from torchmetrics import F1

In [23]:
from torchmetrics import Accuracy

In [24]:
accuracy = Accuracy(num_classes=3)

In [62]:
f1 = F1(num_classes=3,multiclass=True, average='micro')

In [242]:
output = torch.tensor([0.2,0.4,0.3])

In [243]:
output[output.argmax()]

tensor(0.4000)

In [63]:
f1(preds = torch.tensor([[0.3,0.4,0.3],[0.1,0.1,0.8]]) ,target = torch.tensor([[2,2]]))

tensor(0.5000)

In [69]:
x = torch.empty((4,))

In [70]:
y = torch.tensor([1,2])

In [71]:
x[1:3]=y

In [72]:
x

tensor([3.0549e-35, 1.0000e+00, 2.0000e+00, 3.0725e-41])

In [41]:
all_file_train = len(dataloader_train)
murmur_preds_labels_train = torch.empty((all_file_train,))
murmur_preds_probs_train = torch.empty((all_file_train,3))
murmur_labels_train = torch.empty((all_file_train,))
for iter_ in range(args.end_iter):
    start_time = time.time()
    running_loss = 0.0
    running_f1 = 0.0
    running_acc = 0.0
    idx = 0
    cnn.train()
    for log_mel_feature, current_age_group, sex_feature, height_weight, preg_feature, lopc_feature, current_murmur, current_outcome in tqdm.tqdm(dataloader_train):
        log_mel_feature = log_mel_feature.cuda()
        current_age_group = current_age_group.cuda()
        sex_feature = sex_feature.cuda()
        height_weight = height_weight.cuda()
        preg_feature = preg_feature.cuda()
        loc_feature = loc_feature.cuda()
        current_murmur = current_murmur.cuda()
        current_outcome = current_outcome.cuda()
        
        
        
        
        outputs = cnn(log_mel_feature,current_age_group,sex_feature,height_weight, preg_feature,loc_feature)
        loss = criterion(outputs, current_murmur)
        
        
        
        outputs_probs = F.log_softmax(outputs, dim=1)
        
        _, murmur_preds_label_train = torch.max(outputs_probs, dim=1)
        
        
        murmur_preds_label_train = murmur_preds_label_train.data.detach().cpu()

        murmur_preds_labels_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train]= murmur_preds_label_train  #murmur 예측 레이블 (0,1,2)
        
        murmur_preds_probs_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train,:] = outputs_probs
        
        _, murmur_label = torch.max(current_murmur.data, dim=1)  #murmur groud truth (0,1,2)
        
        murmur_labels_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train] = murmur_label.data.detach().cpu()
        
        train_f1 = f1(preds = murmur_preds_labels_train, target = torch.tensor(murmur_labels_train, dtype=torch.int32))
        train_acc = accuracy(preds = murmur_preds_labels_train, target = torch.tensor(murmur_labels_train, dtype=torch.int32))

        idx +=1              

        opt.zero_grad()
        loss.backward()
        opt.step()

        running_loss += loss.item()
        running_f1 += train_f1.item()
        running_acc += train_acc.item()
        
    scheduler.step()
    logger.info("Iteration:{0}, loss = {1:.6f} ,acc = {1:.6f} , f1 = {1:.6f} ".format(iter_, 
                 running_loss/len(dataloader_train),running_acc/len(dataloader_train),running_f1/len(dataloader_train)))
        
    # eval
    all_file_valid = len(dataloader_valid)
    cnn.eval()

    with torch.no_grad():
        running_loss_dev = 0.0
        running_f1_dev = 0.0
        running_acc_dev = 0.0
        index = 0
        murmur_preds_labels = torch.empty((all_file_valid,))
        murmur_preds_prob = torch.empty((all_file_valid,3))
        murmur_labels= torch.empty((all_file_valid,))
        for log_mel_feature, current_age_group, sex_feature, height_weight, preg_feature, loc_feature, current_murmur, current_outcome in tqdm.tqdm(dataloader_valid):
            log_mel_feature_valid = log_mel_feature.cuda()
            current_age_group_valid = current_age_group.cuda()
            sex_feature_valid = sex_feature.cuda()
            height_weight_valid = height_weight.cuda()
            preg_feature_valid = preg_feature.cuda()
            loc_feature_valid = loc_feature.cuda()
            current_murmur_valid = current_murmur.cuda()
            current_outcome_valid = current_outcome.cuda()


            outputs_dev = cnn(log_mel_feature_valid,current_age_group_valid,sex_feature_valid,height_weight_valid,preg_feature_valid,loc_feature_valid)
            loss_dev = criterion(outputs_dev, current_murmur_valid)                   
            running_loss_dev += loss_dev.item()
            
            
            
            
            outputs_dev_probs = F.log_softmax(outputs_dev, dim=1)
            
            _, murmur_preds_label = torch.max(outputs_dev_probs, dim=1)
            
            
            murmur_preds_label = murmur_preds_label.data.detach().cpu()

            murmur_preds_labels[index*args.minibatchsize_valid:index*args.minibatchsize_valid]= murmur_preds_label  #murmur 예측 레이블 (0,1,2)
            
            murmur_preds_probs[index*args.minibatchsize_valid:index*args.minibatchsize_valid,:] = outputs_dev_probs
            
            _, murmur_label = torch.max(current_murmur_valid.data, dim=1)  #murmur groud truth (0,1,2)
            
            murmur_labels[index*args.minibatchsize_valid:index*args.minibatchsize_valid] = murmur_label

            index +=1              
        
            dev_f1 = f1(preds = murmur_preds_labels, target = torch.tensor(murmur_labels, dtype=torch.int32))
            dev_acc = accuracy(preds = murmur_preds_labels,target =  torch.tensor(murmur_labels, dtype=torch.int32))
            
            
            running_f1_dev+= dev_f1.item()
            running_acc_dev+= dev_acc.item()

            

    logger.info("valid Epoch:%d, Train loss=%.4f, Valid loss=%.4f, F1=%.4f, ACC=%.4f" %( iter_,
            running_loss/len(dataloader_train), running_loss_dev/all_file_valid, F1, ACC))
    score_file.write("%d epoch,VALID LOSS %f, VALID F1 %2.2f%% VALID ACC %2.2f%%\n"%(iter_, 
                            running_loss_dev/all_file_valid, running_f1_dev/all_file_valid, running_acc_dev/all_file_valid))
    score_file.flush()
    
    torch.save(cnn.state_dict(), os.path.join(model_path, "{}_{}.model".format(args.model_name, iter_)))
    
    end_time = time.time()
    logger.info("Time used for each epoch training: {} seconds.".format(end_time - start_time))
    logger.info("*" * 50)

100%|███████████████████████████████████████████| 80/80 [00:15<00:00,  5.13it/s]

07-11 21:12 [MainProcess, 21425] [INFO ]  Iteration:0, loss = 0.545619 



100%|████████████████████████████████████████| 631/631 [00:04<00:00, 129.57it/s]

07-11 21:12 [MainProcess, 21425] [INFO ]  valid Epoch:0, Train loss=0.5456, Valid loss=0.5946, F1=0.4064, ACC=0.4040
07-11 21:12 [MainProcess, 21425] [INFO ]  Time used for each epoch training: 20.4787495136261 seconds.
07-11 21:12 [MainProcess, 21425] [INFO ]  **************************************************



100%|███████████████████████████████████████████| 80/80 [00:17<00:00,  4.63it/s]

...

07-11 21:49 [MainProcess, 21425] [INFO ]  valid Epoch:98, Train loss=0.4461, Valid loss=0.5144, F1=0.4755, ACC=0.4594
07-11 21:49 [MainProcess, 21425] [INFO ]  Time used for each epoch training: 23.707907676696777 seconds.
07-11 21:49 [MainProcess, 21425] [INFO ]  **************************************************



100%|███████████████████████████████████████████| 80/80 [00:17<00:00,  4.50it/s]

07-11 21:50 [MainProcess, 21425] [INFO ]  Iteration:99, loss = 0.441957 



100%|████████████████████████████████████████| 631/631 [00:05<00:00, 111.83it/s]

07-11 21:50 [MainProcess, 21425] [INFO ]  valid Epoch:99, Train loss=0.4420, Valid loss=0.5800, F1=0.4794, ACC=0.4956
07-11 21:50 [MainProcess, 21425] [INFO ]  Time used for each epoch training: 23.411577701568604 seconds.
07-11 21:50 [MainProcess, 21425] [INFO ]  **************************************************


In [73]:
all_file_valid

631

In [39]:
torch.tensor(murmur_labels, dtype=torch.int32)

tensor([2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 1, 0, 0, 0, 0, 2, 0,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0,
        0, 2, 2, 0, 1, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 1, 2, 0, 2, 2, 2, 0, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2,
        2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2,
        0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2,
        0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 0, 2, 2,
        2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 1, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2,
        2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2,
        2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 0,

In [308]:
murmur_labels = torch.empty((all_file_valid,3))

In [309]:
murmur_labels[100,:]

tensor([7.9811e-10, 1.8524e+28, 8.9262e-10])

In [ ]:
PRECISION = TP/(TP+FP) 
RECALL = TP/(TP+FN)
F1 = (2*PRECISION*RECALL)/(PRECISION+RECALL)
ACC = (TP+TN)/(TP+FP+TN+FN)

In [15]:
import torch.nn as nn
import torch


class GLU(nn.Module):
    def __init__(self, input_num):
        super(GLU, self).__init__()
        self.sigmoid = nn.Sigmoid()
        self.linear = nn.Linear(input_num, input_num)

    def forward(self, x):
        lin = self.linear(x.permute(0, 2, 3, 1))
        lin = lin.permute(0, 3, 1, 2)
        sig = self.sigmoid(x)
        res = lin * sig
        return res


class ContextGating(nn.Module):
    def __init__(self, input_num):
        super(ContextGating, self).__init__()
        self.sigmoid = nn.Sigmoid()
        self.linear = nn.Linear(input_num, input_num)

    def forward(self, x):
        lin = self.linear(x.permute(0, 2, 3, 1))
        lin = lin.permute(0, 3, 1, 2)
        sig = self.sigmoid(lin)
        res = x * sig
        return res
    
class Dense(nn.Module):
    
    def __init__(self,length):
        super(Dense, self).__init__()
        h1 = nn.Linear(length, 64)
        h2 = nn.Linear(64, 32)
        h3 = nn.Linear(32, 1)
        self.hidden = nn.Sequential(
            h1,
            nn.ReLU(),
            h2,
            nn.ReLU(),
            h3,
        )
        
    def forward(self, x):
        o = self.hidden(x)
        return o.view(-1,1)


class CNN(nn.Module):
    def __init__(
        self,
        n_in_channel=1,
        activation="Relu",
        conv_dropout=0,
        kernel_size=[3, 3, 3, 3, 3, 3],
        padding=[1, 1, 1, 1, 1, 1],
        stride=[1, 1, 1, 1, 1, 1],
        nb_filters=[16, 32, 64, 128, 256, 256,
        pooling=[(2, 2), (2, 2), (1, 2), (1, 2), (1, 2), (1, 2)],
        normalization="batch",
        **transformer_kwargs
    ):
        """
            Initialization of CNN network s
        
        Args:
            n_in_channel: int, number of input channel
            activation: str, activation function
            conv_dropout: float, dropout
            kernel_size: kernel size
            padding: padding
            stride: list, stride
            nb_filters: number of filters
            pooling: list of tuples, time and frequency pooling
            normalization: choose between "batch" for BatchNormalization and "layer" for LayerNormalization.
        """
        super(CNN, self).__init__()

        self.nb_filters = nb_filters
        cnn = nn.Sequential()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(136,68)
        self.fc2 = nn.Linear(68,3)
        self.fc_age = nn.Linear(6,3)
        self.fc_sex = nn.Linear(2,1)
        self.fc_hw = nn.Linear(2,1)
        self.fc_preg = nn.Linear(2,1)
        self.fc_loc = nn.Linear(5,2)
        

        def conv(i, normalization="batch", dropout=None, activ="relu"):
            nIn = n_in_channel if i == 0 else nb_filters[i - 1]
            nOut = nb_filters[i]
            cnn.add_module(
                "conv{0}".format(i),
                nn.Conv2d(nIn, nOut, kernel_size[i], stride[i], padding[i]),
            )
            if normalization == "batch":
                cnn.add_module(
                    "batchnorm{0}".format(i),
                    nn.BatchNorm2d(nOut, eps=0.001, momentum=0.99),
                )
            elif normalization == None:
                pass
                
            elif normalization == "layer":
                cnn.add_module("layernorm{0}".format(i), nn.GroupNorm(1, nOut))

            if activ.lower() == "leakyrelu":
                cnn.add_module("relu{0}".format(i), nn.LeakyReLU(0.2))
            elif activ.lower() == "relu":
                cnn.add_module("relu{0}".format(i), nn.ReLU())
            elif activ.lower() == "glu":
                cnn.add_module("glu{0}".format(i), GLU(nOut))
            elif activ.lower() == "cg":
                cnn.add_module("cg{0}".format(i), ContextGating(nOut))

            if dropout is not None:
                cnn.add_module("dropout{0}".format(i), nn.Dropout(dropout))
            elif dropout is None:
                pass

        # 128x862x64
        for i in range(len(nb_filters)):
            conv(i, normalization=normalization, dropout=conv_dropout, activ=activation)
            cnn.add_module(
                "pooling{0}".format(i), nn.MaxPool2d(pooling[i])
            )  # bs x tframe x mels

        self.cnn = cnn

    def forward(self, mel, age, sex, hw, preg, loc):
        """
        Forward step of the CNN module
        Args:
            x (Tensor): input batch of size (batch_size, n_channels, n_frames, n_freq)
        Returns:
            Tensor: batch embedded
        """
        # conv features
        mel = mel.unsqueeze(1)
        mel = self.cnn(mel)
        mel = self.avgpool(mel)
        mel = mel.view(mel.size(0), -1)
#         print(mel.shape)
        age = self.fc_age(age)
#         print(age.shape)
        sex = self.fc_sex(sex)
#         print(sex.shape)
        hw = self.fc_hw(hw)
#         print(hw.shape)
        preg = self.fc_preg(preg)
#         print(preg.shape)
        loc = self.fc_loc(loc)
#         print(loc.shape)
        x = torch.concat([mel,age,sex,hw,preg,loc],axis=1)
#         print(x.shape)
#         x = self.avgpool(x)
#         x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x =self.fc2(x)
        return x

In [254]:
cnn= CNN()

In [162]:
cnn

CNN(
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Linear(in_features=136, out_features=68, bias=True)
  (fc2): Linear(in_features=68, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
  (fc_age): Linear(in_features=6, out_features=3, bias=True)
  (fc_sex): Linear(in_features=2, out_features=1, bias=True)
  (fc_hw): Linear(in_features=2, out_features=1, bias=True)
  (fc_preg): Linear(in_features=2, out_features=1, bias=True)
  (fc_loc): Linear(in_features=5, out_features=2, bias=True)
  (cnn): Sequential(
    (conv0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm0): BatchNorm2d(16, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    (relu0): ReLU()
    (dropout0): Dropout(p=0, inplace=False)
    (pooling0): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm1): BatchNorm2d(32, eps=0.001, m

In [255]:
x = torch.randn((32,128,600))
age = torch.randn((32,6))
sex = torch.randn((32,2))
hw = torch.randn((32,2))
preg = torch.randn((32,2))
loc = torch.randn((32,5))

In [256]:
output = cnn(x, age, sex, hw, preg, loc)

In [257]:
output.shape

torch.Size([32, 3])

In [271]:
preds = output[0]

In [280]:
torch.max(preds.data)

tensor(0.4683)

In [218]:
output[0]

tensor([0.2681, 0.4115, 0.3204], grad_fn=<SelectBackward0>)

In [122]:
 m = nn.AvgPool2d(3, stride=2)

# m = nn.AvgPool2d((3, 2), stride=(2, 1))
input = torch.randn(20, 16, 50, 32)
output = m(input)

In [123]:
output.shape

torch.Size([20, 16, 24, 15])

In [219]:
torch.ceil(output[0]-0.5).data.cpu().numpy()

array([-0., -0., -0.], dtype=float32)